<a href="https://colab.research.google.com/github/2303A52237/EXPLAINABLE-AI/blob/main/EAI_ASS_7_2237.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [ ]:
data = pd.read_csv('/content/loan_approval.csv')
data.head()

,name,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,Allison Hill,East Jill,113810,389,39698,27,50.0,False
1,Brandon Hall,New Jamesside,44592,729,15446,28,55.0,False
2,Rhonda Smith,Lake Roberto,33278,584,11189,13,45.0,False
3,Gabrielle Davis,West Melanieview,127196,344,48823,29,50.0,False
4,Valerie Gray,Mariastad,66048,496,47174,4,25.0,False


In [ ]:
data.columns

Index(['name', 'city', 'income', 'credit_score', 'loan_amount',
       'years_employed', 'points', 'loan_approved'],
      dtype='object')

In [ ]:
data[data.select_dtypes(include='object').columns] = data.select_dtypes(include='object').apply(lambda x: x.str.strip())

In [ ]:
data['loan_approved'].unique()

array([False,  True])

In [ ]:
print(data.columns.tolist())

['name', 'city', 'income', 'credit_score', 'loan_amount', 'years_employed', 'points', 'loan_approved']


In [ ]:
data = data.drop(columns=["name"])

In [ ]:
# Inspect shape
print("Shape:", data.shape)

Shape: (2000, 7)


In [ ]:
# Inspect datatypes
print("\nData types:\n", data.dtypes)


Data types:
 city               object
income              int64
credit_score        int64
loan_amount         int64
years_employed      int64
points            float64
loan_approved        bool
dtype: object


In [ ]:
# Check missing values
print("\nMissing values:\n", data.isnull().sum())


Missing values:
 city              0
income            0
credit_score      0
loan_amount       0
years_employed    0
points            0
loan_approved     0
dtype: int64


In [ ]:
print(data['loan_approved'].value_counts())

loan_approved
False    1121
True      879
Name: count, dtype: int64


In [ ]:
for col in data.select_dtypes(include="number").columns:
    data[col].fillna(data[col].median(), inplace=True)
for col in data.select_dtypes(include="object").columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

/tmp/ipython-input-531700803.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipython-input-531700803.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
# Encode categorical columns
le = LabelEncoder()
for col in data.select_dtypes(include="object").columns:
    data[col] = le.fit_transform(data[col])

In [ ]:
# Features and target
X = data.drop("loan_approved", axis=1)
y = data["loan_approved"]

In [ ]:
# Scale numeric features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# Handle imbalance with SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
print(y_train.value_counts())       # before SMOTE
print(y_train_res.value_counts())   # after SMOTE

loan_approved
False    897
True     703
Name: count, dtype: int64
loan_approved
False    897
True     897
Name: count, dtype: int64


In [ ]:
# Logistic Regression
lr = LogisticRegression(class_weight="balanced", random_state=42)
lr.fit(X_train_res, y_train_res)

# Random Forest
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# XGBoost
xgb_model = xgb.XGBClassifier(scale_pos_weight=(y_train_res.value_counts()[0]/y_train_res.value_counts()[1]), random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train_res, y_train_res)

/tmp/ipython-input-4094232208.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  xgb_model = xgb.XGBClassifier(scale_pos_weight=(y_train_res.value_counts()[0]/y_train_res.value_counts()[1]), random_state=42, use_label_encoder=False, eval_metric="logloss")
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:57:41] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# Evaluate models
models = {"Logistic Regression": lr, "Random Forest": rf, "XGBoost": xgb_model}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name}")
    print(classification_report(y_test, y_pred))
    print("\n")

Logistic Regression
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



Random Forest
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



XGBoost
              precision    recall  f1-score   support

       False       1.00      1.00      1.00       224
        True       1.00      1.00      1.00       176

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00     

In [ ]:
!pip install dice-ml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.6 MB/s eta 0:00:00


In [ ]:
import dice_ml
from dice_ml.utils import helpers

In [ ]:
# 1. Clean column names (already done in previous steps)
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

# Load the data
data = pd.read_csv('/content/loan_approval.csv')

# 2. Encode categorical columns only
le = LabelEncoder()
for col in ['city']: # Encode 'city' column
    data[col] = le.fit_transform(data[col])

# 3. Split features and target (unscaled)
X = data.drop("loan_approved", axis=1) # Use 'loan_approved' as target
y = data["loan_approved"] # Use 'loan_approved' as target

# Drop the 'name' column as it is not needed for modeling and is not numeric
X = X.drop("name", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 4. SMOTE (optional)
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 5. Train Random Forest on **same features used in DiCE**
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# 6. Now DiCE query instance from X_test will match feature names exactly
query_instance = X_test[y_test==0].iloc[[0]]  # first rejected

In [ ]:
# Imports
import dice_ml
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Fix column spaces (already done in previous steps)

# 2. Define continuous features
continuous_features = [
    'income', 'credit_score', 'loan_amount',
    'years_employed', 'points'
]

# Re-do the data preparation in this cell to ensure consistency for DiCE.
# Load the data again
data_dice = pd.read_csv('/content/loan_approval.csv')

# Drop 'name' column
data_dice = data_dice.drop(columns=["name"])

# Encode categorical columns
le_dice = LabelEncoder()
for col in data_dice.select_dtypes(include="object").columns:
    data_dice[col] = le_dice.fit_transform(data_dice[col])

# Convert the target column to integer type
data_dice['loan_approved'] = data_dice['loan_approved'].astype(int)


# Create DiCE Data object - Use the prepared data_dice DataFrame which includes the target
d = dice_ml.Data(
    dataframe=data_dice, # Use the prepared data_dice DataFrame including the target
    continuous_features=continuous_features,
    outcome_name='loan_approved'
)

# 4. Create DiCE Model object using trained Random Forest
m = dice_ml.Model(model=rf, backend='sklearn')

# 5. Initialize DiCE explainer
exp = dice_ml.Dice(d, m)

# 6. Pick a test instance predicted as negative (Rejected)
#    Use the prepared data_dice and drop target column
#    Need to make sure the index from y_test[y_test==0].index[0] exists in data_dice
neg_idx = y_test[y_test==0].index[0]  # first Rejected from the original split
query_instance = data_dice.loc[[neg_idx]].drop(columns=['loan_approved'])  # only features

# 7. Generate 3 counterfactuals
cf_examples = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite"
)

# 8. Visualize counterfactuals, showing only changed features
cf_examples.visualize_as_dataframe(show_only_changes=True)
cf_df = cf_examples.cf_examples_list[0].final_cfs_df
print("Counterfactuals vs Original:\n", cf_df)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1104' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

Query instance (original outcome : False)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,234,62662,466,26460,25,35.0,False



Diverse Counterfactual set (new outcome: True)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,-,-,-,-,-,58.4,True
1,-,-,-,-,-,92.3,True
2,1122,-,-,-,-,74.2,True


Counterfactuals vs Original:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           466        26460              25    58.4   
1   234   85242           466        26460              25    92.3   
2  1122   62662           466        26460              25    74.2   

   loan_approved  
0           True  
1           True  
2           True  


In [ ]:
query_instance_reindexed = query_instance.iloc[0].reindex(cf_df.columns)
changed_features = cf_df.loc[:, (cf_df != query_instance_reindexed).any()]
influential_features = changed_features.columns.tolist()
print("Most influential features:", influential_features)

Most influential features: ['city', 'income', 'points', 'loan_approved']


In [ ]:
# 8. Check realism/actionability
print("Original instance:\n", query_instance)
print("Counterfactuals:\n", cf_df)

Original instance:
      city  income  credit_score  loan_amount  years_employed  points
795   234   62662           466        26460              25    35.0
Counterfactuals:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           466        26460              25    58.4   
1   234   85242           466        26460              25    92.3   
2  1122   62662           466        26460              25    74.2   

   loan_approved  
0           True  
1           True  
2           True  


In [ ]:
# 9. Generate counterfactuals using Manhattan distance
cf_examples_manhattan = exp.generate_counterfactuals(
    query_instance, total_CFs=3, desired_class="opposite"
)
cf_examples_manhattan.visualize_as_dataframe(show_only_changes=True)
cf_df_manhattan = cf_examples_manhattan.cf_examples_list[0].final_cfs_df
print("Counterfactuals with Manhattan distance:\n", cf_df_manhattan)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '505' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
100%|██████████| 1/1 [00:00<00:00,  1.53it/s]

Query instance (original outcome : False)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,234,62662,466,26460,25,35.0,False



Diverse Counterfactual set (new outcome: True)


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,-,-,-,-,21,93.7,True
1,-,-,-,-,-,99.8,True
2,-,-,-,-,-,85.1,True


Counterfactuals with Manhattan distance:
    city  income  credit_score  loan_amount  years_employed  points  \
0   234   62662           466        26460              21    93.7   
1   234   81593           466        26460              25    99.8   
2   234   62662           466        26460              25    85.1   

   loan_approved  
0           True  
1           True  
2           True  


Reflection How Counterfactual Explanations Improve Trust and Transparency in AI Systems Counterfactual explanations enhance trust and transparency in AI systems by:

Clarifying Decision Logic: They show users how specific changes to input features (e.g., increasing CIBIL score) could alter outcomes (e.g., loan approval), making the model's decision-making process more interpretable. Empowering Users: By providing actionable suggestions (e.g., "increase income by X"), users understand what steps to take, fostering trust in the system’s fairness. Highlighting Model Behavior: Counterfactuals reveal which features are most influential, exposing potential biases or errors in the model, thus improving transparency. Reducing Black-Box Perception: They demystify complex models like Random Forest or XGBoost by presenting intuitive "what-if" scenarios, making AI decisions feel less opaque. Supporting Accountability: By showing how decisions are made and what changes could lead to different outcomes, counterfactuals help stakeholders verify the model aligns with ethical and logical standards.

Real-World Application of Counterfactuals Beyond the Loan Approval Dataset Healthcare Diagnostics:

Use Case: In medical AI systems predicting disease risk (e.g., diabetes), counterfactuals can suggest actionable changes (e.g., "reduce BMI by 5 points" or "lower blood sugar by X") to achieve a healthier outcome. Benefit: Patients and doctors gain insights into critical factors influencing diagnoses, enabling personalized treatment plans and increasing trust in AI-driven medical decisions.